In [1]:
"""
Created on Feb 20 2023
@author: Neven Caplar
@contact: ncaplar@princeton.edu / ncaplar@uw.edu

These comments are theoretically the only ones you need to read to run the notebook

1. Specify the directory in which you want to run the analysis below (PSF_DIRECTORY)
2. Name and place the data in DATA_FOLDER. The data is avaliable at https://github.com/nevencaplar/PFS_Work_In_Progress/tree/master/CutsForTigerMay2
3. TESTING_FOLDER will be filled during the run with images from the analysis analysis

4. (OPTIONAL)Next cell contains some extensions that I use that make life much easier when using jupyter notebook 
    Without them this notebook becomes reallllly huge and hard to deal with
    These can be downloaded from https://github.com/ipython-contrib/jupyter_contrib_nbextensions

"""
############################################################
# name your directory where you want to have files!
PSF_DIRECTORY='/tigress/ncaplar/PFS/'
# place cutouts in this folder - name as you wish
DATA_FOLDER=PSF_DIRECTORY+'TigerAnalysis/CutsForTigerMay2/'
############################################################
    

TESTING_FOLDER=PSF_DIRECTORY+'Testing/'
TESTING_PUPIL_IMAGES_FOLDER=TESTING_FOLDER+'Pupil_Images/'
TESTING_WAVEFRONT_IMAGES_FOLDER=TESTING_FOLDER+'Wavefront_Images/'
TESTING_FINAL_IMAGES_FOLDER=TESTING_FOLDER+'Final_Images/'
import os

for i in [PSF_DIRECTORY,DATA_FOLDER,TESTING_PUPIL_IMAGES_FOLDER,TESTING_WAVEFRONT_IMAGES_FOLDER,TESTING_FINAL_IMAGES_FOLDER]:
    if not os.path.exists(i):
        os.makedirs(i)

In [2]:
# make notebook nice and wide to fill the entire screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

import Zernike_Module
import Zernike_Analysis_Module
from Zernike_Module import *
from Residual_1D_module import * 
from Zernike_Analysis_Module import *

print('Zernike_Module.__version__: '+str(Zernike_Module.__version__))
print('Zernike_Analysis_Module.__version__: '+str(Zernike_Analysis_Module.__version__))

import numpy as np
np.set_printoptions(suppress=True)
np.seterr(divide='ignore', invalid='ignore')
import pandas as pd
import io
import math
import pickle
import glob
import time
from shutil import copy

print(np.__version__)
print(scipy.__version__)
print(pd.__version__)

/tmp/ipykernel_210141/2877485811.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Zernike_Module.__version__: 0.52
Zernike_Analysis_Module.__version__: 0.26l
1.20.3
1.8.1
1.4.2


In [3]:
# First import all of the dataframes contaning information about the spots

import glob

#finalAr_wrong_secondary=np.load("/tigress/ncaplar/ReducedData/Data_May_21_2021/Dataframes/finalAr_Jul2021_wrong_secondary",allow_pickle=True)
finalAr=np.load("/tigress/ncaplar/ReducedData/Data_May_25_2021/Dataframes/finalAr_Jul2021",allow_pickle=True)
finalNe=np.load("/tigress/ncaplar/ReducedData/Data_May_25_2021/Dataframes/finalNe_Jul2021",allow_pickle=True)
finalKr=np.load("/tigress/ncaplar/ReducedData/Data_May_25_2021/Dataframes/finalKr_Jul2021",allow_pickle=True)

def remove(string): 
    return string.replace(" ", "")

We will create scripts for analyzing all of the spots. We have to create scripts for 10-fiber and 21-fiber configuration separatly.

In [10]:
# Select the spots that are well separate (close = 1) and are in 10-fiber configuration
# Id smaller than 120 for Argon, 90 for neon, 40 for Krypton

list_of_Ar_to_analyze=(finalAr[finalAr['close']=='1'].index)[(finalAr[finalAr['close']=='1'].index)<120]
list_of_Ne_to_analyze=finalNe[finalNe['close']=='1'].index [(finalNe[finalNe['close']=='1'].index)<90]
list_of_Kr_to_analyze=finalKr[finalKr['close']==1].index [(finalKr[finalKr['close']==1].index)<40]

#print(len(list_of_HgAr_to_analyze))
print('Number of Argon spots: ' + str(len(list_of_Ar_to_analyze)))
print('Number of Neon spots: ' + str(len(list_of_Ne_to_analyze)))
print('Number of Krypton spots: ' + str(len(list_of_Kr_to_analyze)))

print('Argon index to be analyzed: '+str(list_of_Ar_to_analyze.values))
print('Neon index to be analyzed: '+str(list_of_Ne_to_analyze.values))
print('Krypton index to be analyzed: '+str(list_of_Kr_to_analyze.values))

Number of Argon spots: 80
Number of Neon spots: 40
Number of Krypton spots: 10
Argon index to be analyzed: [  1   3   5   6   7   8   9  11  13  15  17  18  19  20  21  23  25  27
  29  30  31  32  33  35  37  39  41  42  43  44  45  47  49  51  53  54
  55  56  57  59  61  63  65  66  67  68  69  71  73  75  77  78  79  80
  81  83  85  87  89  90  91  92  93  95  97  99 101 102 103 104 105 107
 109 111 113 114 115 116 117 119]
Neon index to be analyzed: [ 2  5  6  7 11 14 15 16 20 23 24 25 29 32 33 34 38 41 42 43 47 50 51 52
 56 59 60 61 65 68 69 70 74 77 78 79 83 86 87 88]
Krypton index to be analyzed: [ 3  7 11 15 19 23 27 31 35 39]


In [ ]:
# Specify the date on which you are running this analysis
# this date will propagate through the names everywhere

date='Nov2221'

# go through all 10 fibers
for f in range(10):
    # select a single spot
    # this will only be used to select the values for offsets when there are 
    # multiple spots - so in this version of the code this is basically irellevant

    # Again, because we are only analyzing spots that are well separated 
    # this is always going to False
    single_spot=list_of_Ar_to_analyze[f*8:(f+1)*8][0]
    if str(finalAr.loc[single_spot]['close'])=='0' or str(finalAr.loc[single_spot]['close'])=='0.5':
        double_source=True
    else:
        double_source=False    
    
    # This a list of all of spots in a single fiber that we analyze
    single_string=" ".join(list_of_Ar_to_analyze[f*8:(f+1)*8].astype(str))
    # Create a script file
    file = open('/home/ncaplar/Scripts/D_Ar_1_'+str(f)+date+'.sh','w') 
    file.write("#!/bin/bash \n")
    # how many nodes are we using when using tiger
    # it has to be 1; is using more than 1 we have to use mpirun, or more advancded methods
    file.write("#SBATCH --nodes=1 # node count \n")
    # how many cored in a node
    file.write("#SBATCH --ntasks-per-node=28 \n") 
    # limit on the time that the process will run
    file.write("#SBATCH --time 23:55:59 \n")
    file.write("#SBATCH --mail-type=begin  \n")
    file.write("#SBATCH --mail-type=end   \n") 
    file.write("#SBATCH --mail-user=ncaplar@princeton.edu \n") 

    # Before running the PFS enviroment needs to be setup - one can do that before submitting the script
    # or it can be specified here 
    # source /tigress/HSC/PFS/stack/current/loadLSST.bash; source scl_source enable devtoolset-6; setup pfs_pipe2d; setup galsim; setup matplotlib; setup display_matplotlib;
    
    # We run Zernike_parameter_estimation
    # We specify observations on which the analyis is being run with -obs 
    #    In this case we specify 34341 34389 34438 which correspodns to one observation at one side of focus (-4 mm), focus (0 mm), and other side of focus (4mm)
    # Specify the number of steps that algorithm will take with nsteps. This is usually as high as you can make it; I usually used 30, 40, or 50 steps
    # Eps specify how many workers will the process spawn - the options are specified in Zernike_parameter_estimation
    # Dataset - make sure that if analyzing 10 fiber configuration that you specify the datasets that correspond to 10 fibers
    # Arc - which lamp are we going to analyze
    # double_sources and double_sources_positions_ratios are populated automatically and should be False and 0,0
    # twentytwo_or_extra: specify how many Zernike parameters will you use (22 or more, i.e., extra)
    # date_of_input - what is the name of the dataframe from which you will supply input 
    # direct_or_interpolation - specify direct
    # date_of_output - name of the output specified earlier
    # analysis_type - it can be defocus or focus. Specify ``defocus'' here. Focus was old mode in which we did not modify any of the Zernike parameters
    # or illumination parameters.
    file.write("python /home/ncaplar/Code/Zernike_parameter_estimation.py -obs 34341 34389 34437 -spot "+str(single_string)+\
               " -nsteps 31 -eps 8 -dataset 6 -arc Ar -double_sources "+str(double_source)+\
               " -double_sources_positions_ratios "+remove(str(list(finalAr.loc[single_spot][['second_offset','second_ratio']].values)))+\
               " -twentytwo_or_extra 56 -date_of_input Sep0521 -direct_or_interpolation direct -date_of_output "+date+" -analysis_type defocus \n")
    file.close()

In [4]:
# How would we do this for 21 fiber configuration?
# We follow the same procedure as above

In [5]:
# Find all of the well isolated spots in 21 fiber configuration
list_of_Ar_to_analyze=(finalAr[finalAr['close']=='1'].index)[(finalAr[finalAr['close']=='1'].index)>120]
list_of_Ne_to_analyze=finalNe[finalNe['close']=='1'].index [(finalNe[finalNe['close']=='1'].index)>90]
list_of_Kr_to_analyze=finalKr[finalKr['close']==1].index [(finalKr[finalKr['close']==1].index)>40]

#print(len(list_of_HgAr_to_analyze))
print(len(list_of_Ar_to_analyze))
print(len(list_of_Ne_to_analyze))
print(len(list_of_Kr_to_analyze))

print(list_of_Ar_to_analyze)
print(list_of_Ne_to_analyze)
print(list_of_Kr_to_analyze)

168
84
21
Int64Index([121, 123, 125, 126, 127, 128, 129, 131, 133, 135,
            ...
            357, 359, 361, 363, 365, 366, 367, 368, 369, 371],
           dtype='int64', length=168)
Int64Index([ 92,  95,  96,  97, 101, 104, 105, 106, 110, 113, 114, 115, 119,
            122, 123, 124, 128, 131, 132, 133, 137, 140, 141, 142, 146, 149,
            150, 151, 155, 158, 159, 160, 164, 167, 168, 169, 173, 176, 177,
            178, 182, 185, 186, 187, 191, 194, 195, 196, 200, 203, 204, 205,
            209, 212, 213, 214, 218, 221, 222, 223, 227, 230, 231, 232, 236,
            239, 240, 241, 245, 248, 249, 250, 254, 257, 258, 259, 263, 266,
            267, 268, 272, 275, 276, 277],
           dtype='int64')
Int64Index([ 43,  47,  51,  55,  59,  63,  67,  71,  75,  79,  83,  87,  91,
             95,  99, 103, 107, 111, 115, 119, 123],
           dtype='int64')


In [ ]:
date='Nov2221'

# note the difference here in the number of fibers
# I specify here all 21 fibers, even though I know that 2 fibers will fail immedietly
# These prcoess will just die quickly so there is no significant overhead
for f in range(21):
    single_spot=list_of_Ar_to_analyze[f*8:(f+1)*8][0]
    if str(finalAr.loc[single_spot]['close'])=='0' or str(finalAr.loc[single_spot]['close'])=='0.5':
        double_source=True
    else:
        double_source=False    
        
    single_string=" ".join(list_of_Ar_to_analyze[f*8:(f+1)*8].astype(str))
    # I name these scripts differently - `D_Ar_2` instead of `D_Ar_1` above
    file = open('/home/ncaplar/Scripts/D_Ar_2_'+str(f)+date+'.sh','w') 
    file.write("#!/bin/bash \n")
    file.write("#SBATCH --nodes=1 # node count \n")
    file.write("#SBATCH --ntasks-per-node=28 \n") 
    file.write("#SBATCH --time 23:55:59 \n")
    file.write("#SBATCH --mail-type=begin  \n")
    file.write("#SBATCH --mail-type=end   \n") 
    file.write("#SBATCH --mail-user=ncaplar@princeton.edu \n") 

    file.write("\n")
    file.write("#1. Observation (e.g., 8567) \n") 
    file.write("#2. Threads \n") 
    file.write("#3. Steps \n") 
    file.write("\n")
    
    # Note the difference in 1. observations specified and 2. dataset variable!
    file.write("python /home/ncaplar/Code/Zernike_parameter_estimation.py -obs 51485 51581 51677 -spot "+str(single_string)+\
               " -nsteps 31 -eps 8 -dataset 8 -arc Ar -double_sources "+str(double_source)+\
               " -double_sources_positions_ratios "+remove(str(list(finalAr.loc[single_spot][['second_offset','second_ratio']].values)))+\
               " -twentytwo_or_extra 56 -date_of_input Sep0521 -direct_or_interpolation direct -date_of_output "+date+" -analysis_type defocus \n")
    file.close()    

In [6]:
# to submit such a script on tiger and or Hilo machine
# one would then run sbatch e.g., `/home/ncaplar/Scripts/D_Kr_2_15Nov2221.sh` where `/home/ncaplar/Scripts/` is the location where the script have been stored
# example of the output during the run: https://gist.github.com/nevencaplar/bba8b38307ad4f8ef39ea43b3eef7229